<a href="https://colab.research.google.com/github/RafaelSilvaGodoy/PosCienciaDeDados/blob/master/Aula2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gitly

     |████████████████████████████████| 13.1MB 321kB/s 
     |████████████████████████████████| 74.0MB 50kB/s 
  Created wheel for gitly: filename=gitly-1.1.4-cp36-none-any.whl size=4523 sha256=4cb045a3f0e70ac6c43c2940da15037ab78839e8da5b3b522804d0bde90ddb19
  Stored in directory: /root/.cache/pip/wheels/41/5c/0b/6cd3a688991c83f17c948adf0775f7907a0406d1119bd809af
Successfully built gitly
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
from gitly.colab.plot import GitlyPlotter
import pandas as pd

gitly = GitlyPlotter('git')
gitly.config_render(renderer='colab')

In [ ]:
df_clientes = pd.read_csv('/content/drive/My Drive/Colab Notebooks/POS_SEMESTRE1/Ciência de Dados/Aula1/olist_customers_dataset.csv')
df_geo = pd.read_csv('/content/drive/My Drive/Colab Notebooks/POS_SEMESTRE1/Ciência de Dados/Aula1/olist_geolocation_dataset.csv', encoding='utf8')
df_compra_itens = pd.read_csv('/content/drive/My Drive/Colab Notebooks/POS_SEMESTRE1/Ciência de Dados/Aula1/olist_order_items_dataset.csv')
df_compra_pagamentos = pd.read_csv('/content/drive/My Drive/Colab Notebooks/POS_SEMESTRE1/Ciência de Dados/Aula1/olist_order_payments_dataset.csv')
df_reviews = pd.read_csv('/content/drive/My Drive/Colab Notebooks/POS_SEMESTRE1/Ciência de Dados/Aula1/olist_order_reviews_dataset.csv')
df_compras = pd.read_csv('/content/drive/My Drive/Colab Notebooks/POS_SEMESTRE1/Ciência de Dados/Aula1/olist_orders_dataset.csv')
df_produtos = pd.read_csv('/content/drive/My Drive/Colab Notebooks/POS_SEMESTRE1/Ciência de Dados/Aula1/olist_products_dataset.csv')
df_vendedores = pd.read_csv('/content/drive/My Drive/Colab Notebooks/POS_SEMESTRE1/Ciência de Dados/Aula1/olist_sellers_dataset.csv')

# Clientes

In [ ]:
print(df_clientes.head(3))
print()
print(df_clientes.shape)
print()
print(df_clientes.nunique())

                        customer_id  ... customer_state
0  06b8999e2fba1a1fbc88172c00ba8bc7  ...             SP
1  18955e83d337fd6b2def6b18a428ac77  ...             SP
2  4e7b3e00288586ebd08712fdd0374a03  ...             SP

[3 rows x 5 columns]

(99441, 5)

customer_id                 99441
customer_unique_id          96096
customer_zip_code_prefix    14994
customer_city                4119
customer_state                 27
dtype: int64


In [ ]:
df_clientes.agg(['nunique','count','size'])

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
nunique,99441,96096,14994,4119,27
count,99441,99441,99441,99441,99441
size,99441,99441,99441,99441,99441


In [ ]:
print('Quantidade de clientes únicos',df_clientes.customer_unique_id.nunique() )

Quantidade de clientes únicos 96096


In [ ]:
df_clientes_unique = df_clientes.drop_duplicates('customer_unique_id')
df_clientes_unique.shape

(96096, 5)

In [ ]:
df_top5_estados = df_clientes_unique.value_counts('customer_state').head(5)
print(df_top5_estados)

customer_state
SP    40295
RJ    12377
MG    11255
RS     5277
PR     4882
dtype: int64


In [ ]:
import plotly.express as px

fig = px.bar(df_top5_estados, labels={'customer_state': 'Número de Clientes', 'index':'Estado ()'}, title = '5 Estados com mais clientes')
gitly.show( fig )

In [ ]:
df_top5_cidades = df_clientes_unique.value_counts('customer_city').head(5)
print(df_top5_cidades)

customer_city
sao paulo         14971
rio de janeiro     6611
belo horizonte     2671
brasilia           2066
curitiba           1462
dtype: int64


In [ ]:
fig = px.bar(df_top5_cidades, labels={'customer_city': 'Número de Clientes', 'index':'Cidade ()'}, title = '5 Cidades com mais clientes')
gitly.show( fig )

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, column_titles = ['Cidades com mais clientes','Estados com mais clientes'])

fig.add_bar(x=df_top5_cidades.index , y=df_top5_cidades, row = 1, col = 1)
fig.add_bar(x=df_top5_estados.index, y=df_top5_estados, row = 1, col = 2)

fig.update_layout(dict({'showlegend': False, 'title':'Clientes: Cidades vs Estados'}))
gitly.show(fig)

In [ ]:
df_top4_bairros = df_clientes_unique.customer_zip_code_prefix.value_counts().head(4).rename('qtd_clientes_bairro').reset_index()
df_top4_bairros

,index,qtd_clientes_bairro
0,22790,136
1,22793,119
2,24220,113
3,24230,112


In [ ]:
!pip install pycep_correios

     |████████████████████████████████| 102kB 3.2MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 


In [ ]:
# fonte https://imasters.com.br/back-end/python-consulta-de-cep-com-pycepcorreios
import pycep_correios

def bairro_cidade( cep ):
    end = pycep_correios.consultar_cep( str( cep ) + '000' )
    return '{} / {}'.format(end['bairro'], end['cidade'])

df_top4_bairros['distrito'] = df_top4_bairros['index'].apply( bairro_cidade )
df_top4_bairros

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning:

Call to deprecated function (or staticmethod) consultar_cep. ('consultar_cep' is no longer supported and will be removed in a future release. Please, use 'get_address_from_cep' instead.) -- Deprecated since version 4.0.0.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning:

Call to deprecated function (or staticmethod) consultar_cep. ('consultar_cep' is no longer supported and will be removed in a future release. Please, use 'get_address_from_cep' instead.) -- Deprecated since version 4.0.0.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning:

Call to deprecated function (or staticmethod) consultar_cep. ('consultar_cep' is no longer supported and will be removed in a future release. Please, use 'get_address_from_cep' instead.) -- Deprecated since version 4.0.0.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationW

,index,qtd_clientes_bairro,distrito
0,22790,136,Recreio dos Bandeirantes / Rio de Janeiro
1,22793,119,Barra da Tijuca / Rio de Janeiro
2,24220,113,Icaraí / Niterói
3,24230,112,Icaraí / Niterói


# Produtos

In [ ]:
print(df_produtos.shape)
print()
print(df_produtos.nunique())
print()
df_produtos.head(3)

(32951, 9)

product_id                    32951
product_category_name            73
product_name_lenght              66
product_description_lenght     2960
product_photos_qty               19
product_weight_g               2204
product_length_cm                99
product_height_cm               102
product_width_cm                 95
dtype: int64



,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0


In [ ]:
df_produtos.agg(['nunique','count','size'])

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
nunique,32951,73,66,2960,19,2204,99,102,95
count,32951,32341,32341,32341,32341,32949,32949,32949,32949
size,32951,32951,32951,32951,32951,32951,32951,32951,32951


In [ ]:
df_produtos.product_category_name.fillna(value='nao informado', inplace = True)
df_produtos.dropna(inplace = True)
df_produtos.agg(['nunique','count','size'])

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
nunique,32340,73,66,2960,19,2202,99,102,95
count,32340,32340,32340,32340,32340,32340,32340,32340,32340
size,32340,32340,32340,32340,32340,32340,32340,32340,32340


In [ ]:
df_categorias_mais_vendidas = df_produtos.product_category_name.value_counts().head(5)
df_categorias_mais_vendidas

cama_mesa_banho          3029
esporte_lazer            2867
moveis_decoracao         2657
beleza_saude             2444
utilidades_domesticas    2335
Name: product_category_name, dtype: int64

In [ ]:
import numpy as np

df_peso_medio_produtos = df_produtos.groupby(by='product_category_name').mean()['product_weight_g']
df_peso_medio_produtos

product_category_name
agro_industria_e_comercio    5263.405405
alimentos                     923.414634
alimentos_bebidas            1307.932692
artes                        1691.763636
artes_e_artesanato           1164.578947
                                ...     
sinalizacao_e_seguranca      2969.817204
tablets_impressao_imagem      381.444444
telefonia                     236.506173
telefonia_fixa                660.827586
utilidades_domesticas        3020.793576
Name: product_weight_g, Length: 73, dtype: float64

In [ ]:
df_mais_vendido_peso_medio = pd.merge(left= df_categorias_mais_vendidas, right = df_peso_medio_produtos, how='left', left_index = True, right_index = True)
df_mais_vendido_peso_medio

,product_category_name,product_weight_g
cama_mesa_banho,3029,2456.405084
esporte_lazer,2867,2024.670736
moveis_decoracao,2657,3008.266842
beleza_saude,2444,1434.793781
utilidades_domesticas,2335,3020.793576


# Compras

In [ ]:
print(df_compras.shape)
print()
print(df_compras.nunique())
print()
df_compras.head(3)

(99441, 8)

order_id                         99441
customer_id                      99441
order_status                         8
order_purchase_timestamp         98875
order_approved_at                90733
order_delivered_carrier_date     81018
order_delivered_customer_date    95664
order_estimated_delivery_date      459
dtype: int64



,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00


In [ ]:
df_compras.agg(['nunique','count','size'])

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
nunique,99441,99441,8,98875,90733,81018,95664,459
count,99441,99441,99441,99441,99281,97658,96476,99441
size,99441,99441,99441,99441,99441,99441,99441,99441


In [ ]:
df_compras.fillna(method='ffill', inplace = True)
df_compras.agg(['nunique','count','size'])

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
nunique,99441,99441,8,98875,90733,81018,95664,459
count,99441,99441,99441,99441,99441,99441,99441,99441
size,99441,99441,99441,99441,99441,99441,99441,99441


In [ ]:
 df_status = df_compras.order_status.value_counts()
 df_status

delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

In [ ]:
fig = px.bar(df_status, y='order_status', title='Status do pedido', log_y=True, labels={'order_status': 'Pedidos', 'index':'Status'})
gitly.show( fig )

In [ ]:
df_compras_dt = df_compras[['order_id','customer_id','order_status']].copy()
df_compras_dt.head(3)

,order_id,customer_id,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered


In [ ]:
pd.to_datetime(df_compras['order_purchase_timestamp'], format='%Y-%m-%d')

0       2017-10-02 10:56:33
1       2018-07-24 20:41:37
2       2018-08-08 08:38:49
3       2017-11-18 19:28:06
4       2018-02-13 21:18:39
                ...        
99436   2017-03-09 09:54:05
99437   2018-02-06 12:58:58
99438   2017-08-27 14:46:43
99439   2018-01-08 21:28:27
99440   2018-03-08 20:57:30
Name: order_purchase_timestamp, Length: 99441, dtype: datetime64[ns]

In [ ]:
lista_colunas_datas = list(df_compras.iloc[:1, 3:].columns)

for coluna in lista_colunas_datas:
  df_compras_dt[coluna] = pd.to_datetime(df_compras[coluna], format='%Y-%m-%d')

df_compras_dt.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04


In [ ]:
np.mean(df_compras_dt.order_delivered_customer_date - df_compras_dt.order_estimated_delivery_date)

Timedelta('-10 days +11:52:58.261491731')

In [ ]:
em_tempo = (df_compras_dt.order_delivered_customer_date <= df_compras_dt.order_estimated_delivery_date).sum()
total = (df_compras_dt.order_delivered_customer_date <= df_compras_dt.order_estimated_delivery_date).count()
atrasados = total - em_tempo

fig = px.pie( values=[em_tempo, atrasados], names=['Dentro do Estimado','Atrasados'], title='Eficiência na Entrega')
fig.show()

In [ ]:
df_compras_dt['delivery_interval'] = df_compras_dt.order_delivered_customer_date - df_compras_dt.order_purchase_timestamp

df_compras_dt['numeric_month'] =  df_compras_dt.order_purchase_timestamp.dt.month
df_compras_dt['month'] =  df_compras_dt.order_purchase_timestamp.dt.month_name()
df_compras_dt['year'] =  df_compras_dt.order_purchase_timestamp.dt.year

df_compras_dt['days_delivery_interval'] = df_compras_dt.delivery_interval.dt.days

df_compras_mes =  df_compras_dt[['numeric_month','year','days_delivery_interval' ]].groupby( by='numeric_month' ).apply(np.mean)


fig = px.bar(df_compras_mes, y='days_delivery_interval', title='Média de prazo de entrega', labels={'days_delivery_interval': 'Dias decorridos', 'index':'Mês'})
gitly.show( fig )

In [ ]:
df_n_compras_mes = df_compras_dt.numeric_month.value_counts()
df_n_compras_mes_plot = df_n_compras_mes.reset_index().sort_values(by='index', ascending=True)
df_n_compras_mes_plot

,index,numeric_month
7,1,8069
6,2,8508
3,3,9893
5,4,9343
1,5,10573
4,6,9412
2,7,10318
0,8,10843
11,9,4305
10,10,4959


# Compras - Itens


In [ ]:
df_compra_itens.shape

(112650, 7)

In [ ]:
df_compra_itens.agg(['nunique','count','size'])

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
nunique,98666,21,32951,3095,93318,5968,6999
count,112650,112650,112650,112650,112650,112650,112650
size,112650,112650,112650,112650,112650,112650,112650


In [ ]:
df_compra_itens.head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87


Perguntas:
- Qual o produto mais vendido?
- **O frete do produto mais vendido muda de acordo com a região?**
- Qual o frete médio do produto mais vendido?
- Qual o top 5 vendedores?
- Quais produtos que os top 5 vendedores mais vendem?
- Variação de preço ao longo do tempo.
- Qual produto fatura mais?
- Venda de produto x Ano (top 5?)
- Volume de dinheiro em transações ao longo dos meses.
----------------------------------------------
Responder
- Qual o produto mais vendido?
- Qual o frete médio do produto mais vendido?
- Qual produto fatura mais?
- Qual o top 5 vendedores?
----------------------------------------------
Para casa:
- Variação de preço ao longo do tempo.
- Venda de produto x Ano (top 5?)
- Volume de dinheiro em transações ao longo dos meses.

In [ ]:
produto_max = df_compra_itens.groupby(by='product_id').sum()['order_item_id'].sort_values(ascending=False).head(1)
produto_max

product_id
422879e10f46682990de24d770e7f83d    793
Name: order_item_id, dtype: int64

In [ ]:
aux = df_compra_itens[df_compra_itens['product_id'] == produto_max.index[0]]
frete_max = aux.freight_value.mean()
frete_max

15.752148760330588

In [ ]:
df_compra_itens['price_total'] = df_compra_itens.order_item_id * df_compra_itens.price
fatura_max = df_compra_itens.groupby(by='product_id').apply(sum)[['price_total', 'order_item_id']].sort_values(by ='price_total',ascending=False).head(5)
fatura_max

,price_total,order_item_id
product_id,,
bb50f2e236e5eea0100680137654686c,70485.00,215
5769ef0a239114ac3a854af00df129e4,60480.00,36
6cdd53843498f92890544667809f1595,57557.60,164
d1c427060a0f73f6b889a5c7c61f2ac4,50940.39,369
d6160fb7873f184099d9bc95e30376af,48899.34,35


In [ ]:
top5_vend =  df_compra_itens.groupby(by = 'seller_id').apply(sum)[['price_total', 'order_item_id']].sort_values(by ='price_total',ascending=False).head(5)
top5_vend

,price_total,order_item_id
seller_id,,
7c67e1448b00f6e969d365cea6b010ab,292489.30,2172
53243585a1d6dc2643021fd1853d8905,244941.39,504
4869f7a5dfa277a7dca6462dcf3b52b2,235628.51,1193
4a3ca9315b744ce9f8e9374361493884,226871.72,2269
da8622b14eb17ae2831f4ac5b9dab84a,197382.15,1888


In [ ]:
# top 5 categorias mais vendidas

print(df_produtos.columns)
print(df_compra_itens.columns)

Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')
Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value', 'price_total'],
      dtype='object')


In [ ]:
df_merge = pd.merge(left=df_compra_itens, right=df_produtos, on='product_id', how='left')
df_merge.head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,price_total,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,58.9,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93,239.9,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87,199.0,moveis_decoracao,59.0,695.0,2.0,3050.0,33.0,13.0,33.0


In [ ]:
top5_cat = df_merge.groupby('product_category_name').sum().order_item_id.sort_values(ascending=False).head()
top5_cat

product_category_name
cama_mesa_banho           13665
moveis_decoracao          11540
beleza_saude              11081
esporte_lazer              9932
informatica_acessorios     9874
Name: order_item_id, dtype: int64

In [ ]:
fig = px.bar(top5_cat, y='order_item_id', title='Top 5 Categorias', labels={'order_item_id': 'Qtd. Vendidas', 'product_category_name':'Categoria'})
gitly.show( fig )

In [ ]:
top5_cat_preco = df_merge.groupby('product_category_name').sum().price_total.sort_values(ascending=False).head()
top5_cat_preco

product_category_name
beleza_saude              1347468.49
relogios_presentes        1259634.58
cama_mesa_banho           1228795.46
informatica_acessorios    1135454.64
esporte_lazer             1082435.42
Name: price_total, dtype: float64